In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import torch

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
#np.seterr(divide='print', invalid='print')
from scipy.sparse import hstack, csr_matrix
from xgboost import XGBClassifier
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import IPython
from plotly.offline import init_notebook_mode
%matplotlib inline
import plotly.offline as offline
from sklearn.calibration import CalibratedClassifierCV
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from prettytable import PrettyTable
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pickle


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [24]:
# Load train and test data
train_dataset = pd.read_csv("dataset/UNSW_NB15_training-set.csv", encoding = "ISO-8859-1")
train_dataset.head()

,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [28]:
test_dataset = pd.read_csv("dataset/UNSW_NB15_testing-set.csv", encoding = "ISO-8859-1")
test_dataset.head()

,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(train_dataset.drop(['attack_cat', 'label'], axis=1), train_dataset.label, stratify=train_dataset.label, test_size=0.20)
X_test, y_test = test_dataset.drop(['attack_cat', 'label'], axis=1), test_dataset.label

train_df = pd.read_csv(train_file, header=None, dtype=str)
test_df = pd.read_csv(test_file, header=None, dtype=str)
# print(train_df)
# # check if there are any samples in the dataframe
if train_df.empty:
    raise ValueError("Training dataframe is empty")

# check if there are any samples in the dataframe
if test_df.empty:
    raise ValueError("Testing dataframe is empty")


In [ ]:
# Check for non-numeric values in the training dataframe
train_df_numeric = train_df.apply(pd.to_numeric, errors='coerce')
if train_df_numeric.isna().all().all():
    raise ValueError("No numeric samples in the training dataframe")

print(train_df)

In [ ]:
if train_df.empty:
    raise ValueError("No numeric samples in the training dataframe")    
elif train_df_numeric.isna().all().all():
    raise ValueError("No numeric samples in the training dataframe after dropping non-numeric rows")
print(train_df)

In [ ]:
# Combine train and test data for label encoding
#print(train_df)
#print("ouavbouqebovb")
combined_df = pd.concat([train_df, test_df])
print(combined_df)
#print(combined_df)

# Drop rows with non-numeric values
combined_df = combined_df.apply(pd.to_numeric, errors='coerce').dropna()
# #print(combined_df)
# train_df = train_df.loc[combined_df.index]
# test_df = test_df.loc[combined_df.index]

In [ ]:
# # check if there are any samples in the dataframe after dropping non-numeric rows
if train_df.empty:
    raise ValueError("No numeric samples in the training dataframe")    
elif train_df_numeric.isna().all().all():
    raise ValueError("No numeric samples in the training dataframe after dropping non-numeric rows")


In [ ]:
# Label encode categorical features
categorical_features = [1, 2, 3, 41]
for feature in categorical_features:
    encoder = LabelEncoder()
    encoder.fit(combined_df[feature].astype(str))
    train_df[feature] = encoder.transform(train_df[feature])
    test_df[feature] = encoder.transform(test_df[feature])


In [ ]:
# Scale numerical features
# scaler = StandardScaler()
scaler = MinMaxScaler()
numerical_features = [0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

if not train_df.empty:
    train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])
if not test_df.empty:
    test_df[numerical_features] = scaler.transform(test_df[numerical_features])
# train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])
# test_df[numerical_features] = scaler.transform(test_df[numerical_features])


In [ ]:
# Split dataset into features and labels
X_train = train_df.drop([41, 42], axis=1).values
y_train = train_df[41].values
X_test = test_df.drop([41, 42], axis=1).values
y_test = test_df[41].values

In [ ]:
# Convert data to PyTorch tensors
# X_train = torch.tensor(X_train, dtype=torch.float32)
X_train = torch.tensor(X_train.astype('float32'))
y_train = torch.tensor(y_train, dtype=torch.long)
# X_test = torch.tensor(X_test, dtype=torch.float32)
X_test = torch.tensor(X_test.astype('float32'))
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
X_train 
# y_train 
# X_test
# y_test